# Filter Table Rows by Attributes

<div class="alert alert-block alert-info">
<b>Tip:</b> Find the full source code and run TagsHasher on Jina Hub: https://hub.jina.ai.</div>



Big news, you can use Jina to filter table rows by their attributes! Such an amazing feature that only exists since... 47 years ago, aka SQL!\
Jina as a neural search framework surely won't implement a SQL database from scratch. The question here is: is it possible to leverage what we learned about neural search (embedding, indexing, nearest-neighbour matching) to enable similar feature like SQL, e.g. filter, select?

Yes! Jina can do this. In this article, I will give you a walkthrough on how to filter the tabular data using Jina and without SQL (also no GPT-3). Let's call this mini-project the neuretro-SQL.

## Feature hashing

The first thing you want to learn is feature hashing.

In general, feature hashing is a great way to embed **unbounded** number of features into fixed-size vectors. We will leverage the same idea here to embed the columns of the tabular data into fixed-size vectors.

## Load CSV as DocumentArray

Let's look at an example CSV file. Here I use a [film dataset](https://perso.telecom-paristech.fr/eagan/class/igr204/data/film.csv) that looks like the following:  

![film dataset](film-dataset.png)

Let's load the data from the web and put them into a DocumentArray:

In [1]:
import io

from docarray import Document, DocumentArray

# Load some online CSV file dataset
src = Document(
    uri='https://perso.telecom-paristech.fr/eagan/class/igr204/data/film.csv'
).load_uri_to_text('iso8859')
da = DocumentArray.from_csv(io.StringIO(src.text), dialect='auto')

Here we use Document API to download the data, convert it into the right charset, and load it via our CSV API as a DocumentArray.

Looks like we got 1660 Documents in total, let's take one sample from it and take a look:

In [ ]:
print(da[5].to_json())

It looks like this Document has two important non-empty attributes `id` and `tags`, and all values in `tags` correspond to the column value we have in the CSV data.\
Now our task is clear: we want to filter Documents from this DocumentArray according to their values in `.tags`, but no SQL, pure Jina, pure neural search.

## Embed columns as vectors

To embed columns into vectors, we first notice that each "column-item" in `.tags` is actually a `Tuple[str, Any]` pair. The first part, a string, represents the column title, e.g. "Actor", "Actress", "Director".\
To hash such values, you can use the following function:

In [ ]:
import hashlib

h = lambda x: int(hashlib.md5(str(x).encode('utf-8')).hexdigest(), base=16) % 256

h('Actor')
h('Director')
h('Length')

Now that we have indices, the actual value on that index, namely the `Any` part of that `Tuple[str, Any]` pair needs some extra thought.

First, some values are numbers like integers or floats, they are a good hash by themselves, so they do not need another hash function.\
Boolean values are the same, 0 and 1 are pretty representative. Strings can be handled in the same way above. What about lists, tuples and dicts?\
We can serialize them into JSON strings and then apply our string hash.

The final hash function looks like the following:


In [5]:
def _any_hash(self, v):
    try:
        return int(v)  # parse int parameter
    except ValueError:
        try:
            return float(v)  # parse float parameter
        except ValueError:
            if not v:
                # ignore it when the parameter is empty
                return 0
            if isinstance(v, str):
                v = v.strip()
                if v.lower() in {'true', 'yes'}:  # parse boolean parameter
                    return 1
                if v.lower() in {'false', 'no'}:
                    return 0
            if isinstance(v, (tuple, dict, list)):
                v = json.dumps(v, sort_keys=True)

    return int(self.hash(str(v).encode('utf-8')).hexdigest(), base=16)

If you apply this directly, you will get extremely big integers on the embedding values.
So big that you don't even want to look at or store it (for numerical and stability reason).

So we need to bound it.\
We can introdoce the variables `n_dim` and `max_val` to bound the dimension of our emeddings "horizontally" and "vertically", respectively:

In [6]:
n_dim: int = 256
max_val: int = 65536

Here we give a larger number to `max_val` then to `n_dim`.\
This is because the likelihood of a collision happens on vertical direction is in general much higher than on horizontal direction (otherwise, it implies there are more variants on the column name than on the column value, which then suggests the table-maker to simply "transpose" the whole table for better readability).

The final embedding procedure is then very simple:

In [11]:
def encode(self, docs: DocumentArray, **kwargs):
    for idx, doc in enumerate(docs):
        if doc.tags:
            idxs, data = [], []  # sparse
            table = np.zeros(self.n_dim)  # dense
            for k, v in doc.tags.items():
                h = self._any_hash(k)
                sign_h = np.sign(h)
                col = h % self.n_dim
                val = self._any_hash(v)
                sign_v = np.sign(val)
                val = val % self.max_val
                idxs.append((0, col))
                val = sign_h * sign_v * val
                data.append(val)
                table[col] += val

            if self.sparse:
                doc.embedding = csr_matrix(
                    (data, zip(*idxs)), shape=(1, self.n_dim)
                )
            else:
                doc.embedding = table

## Put it all together

Now you can put your code together into one Executor:

In [17]:
from docarray import Document, DocumentArray
from jina import Executor, requests
import hashlib
import numpy as np

class TagsHasher(Executor):
    def __init__(self, n_dim: int = 256, max_val: int = 65536, sparse: bool = False, **kwargs):
        super().__init__(**kwargs)
        self.n_dim = n_dim
        self.max_val = max_val
        self.hash = hashlib.md5
        self.sparse = sparse
        
    def _any_hash(self, v):
        try:
            return int(v)  # parse int parameter
        except ValueError:
            try:
                return float(v)  # parse float parameter
            except ValueError:
                if not v:
                    # ignore it when the parameter is empty
                    return 0
                if isinstance(v, str):
                    v = v.strip()
                    if v.lower() in {'true', 'yes'}:  # parse boolean parameter
                        return 1
                    if v.lower() in {'false', 'no'}:
                        return 0
                if isinstance(v, (tuple, dict, list)):
                    v = json.dumps(v, sort_keys=True)
        return int(self.hash(str(v).encode('utf-8')).hexdigest(), base=16)
        
    @requests
    def encode(self, docs: DocumentArray, **kwargs):
        if self.sparse:
            from scipy.sparse import csr_matrix
            
        for idx, doc in enumerate(docs):
            if doc.tags:
                idxs, data = [], []  # sparse
                table = np.zeros(self.n_dim)  # dense
                for k, v in doc.tags.items():
                    h = self._any_hash(k)
                    sign_h = np.sign(h)
                    col = h % self.n_dim
                    val = self._any_hash(v)
                    sign_v = np.sign(val)
                    val = val % self.max_val
                    idxs.append((0, col))
                    val = sign_h * sign_v * val
                    data.append(val)
                    table[col] += val

                if self.sparse:
                    doc.embedding = csr_matrix(
                        (data, zip(*idxs)), shape=(1, self.n_dim)
                    )
                else:
                    doc.embedding = table
    

Let's encode our loaded DocumentArray:

In [18]:
th = TagsHasher()
th.encode(da)

Now let's build some filters as Document:

In [19]:
filters = [
    {"Subject": "Comedy"},
    {"Year": 1987},
    {"Subject": "Comedy", "Year": 1987}
]

qa = DocumentArray([Document(tags=f) for f in filters])

Encode the filter with `TagsHasher` to get the embeddings.

In [20]:
th.encode(qa)

Now that we have embeddings for both indexed docs `da` (i.e. our film CSV table), and the query docs `qa` (our filters), we can use `.match` function to find nearest neighbours.


In [21]:
qa.match(da, limit=5, exclude_self=True, metric='jaccard', use_scipy=True)

Note that here I use Jaccard distance instead of the  cosine distance. This is because the closeness of the value on each feature is meaningless, as the value is the result of a hash function. Whereas in `FeatureHashser`'s example, the value represents the term frequency of a word, so it was meaningful there. This needs to be kept in mind when using `TagsHasher`.

Finally, let's see some results. Here I only print top-5 matches.

In [ ]:
for d in qa:
    print('my filter is:', d.tags)
    for m in d.matches:
        print(m.tags)